In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 147k/84.1M [00:00<01:03, 1.32MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 64.9MB/s]


In [0]:
TEXT.build_vocab(train_data, max_size = 10000,vectors = "glove.6B.300d", unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [0]:
BATCH_SIZE = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [85]:
print(device)

cuda


In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = embedding_dim, out_channels = 150, kernel_size = fs)for fs in [4,5]])
        self.conv2 = nn.Conv1d(in_channels = 1, out_channels = 100, kernel_size = 3)              
        self.fc1 = nn.Linear(200, 100) 
        #self.fc1_bn=nn.BatchNorm1d(100)
        self.fc2=nn.Linear(100,1)
        #self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        # print(text.size())
        embedded = self.embedding(text)        
        embedded = embedded.permute(0, 2, 1)

        sent_len=embedded.size(2)
        padding=3000-sent_len
        batch_size=embedded.size(0)
        torch_padding=torch.zeros(batch_size,300,padding,dtype = embedded.dtype,device = embedded.device)
        lz=[embedded,torch_padding]
        zcat = torch.cat(lz, dim = 2)

        conved = [F.relu(conv(zcat)) for conv in self.convs]

        pooled=[]
        for c in conved:
          pooled.append(F.max_pool1d(c,c.shape[2]))

        pooled = [f.permute(0,2,1) for f in pooled]

        pooled2 = [F.max_pool1d(p, 2) for p in pooled]

        pooled3 = [F.relu(self.conv2(p1)) for p1 in pooled2]

        pooled4=[]
        for c in pooled3:
            pooled4.append(F.max_pool1d(c,c.shape[2]))
        # for f in pooled4:
        #   print(f.size(),"Pooled4")
        final = torch.cat(pooled4,dim = 1)
        #print(final.size(),"final")
        final = final.reshape(batch_size,200)
        full1 = self.fc1(final)
        full2= self.fc2(full1)
        return full2



        # cat = torch.cat(pooled, dim = 2)
        # conved2 = F.relu(self.convs2(cat)) 
        # pooled2 = F.max_pool1d(conved2, 2)
        # pooled2=pooled2.reshape(batch_size,149700)
        # full1= self.fc1(pooled2)
        # full1_bn=self.dropout(self.fc1_bn(full1))
        # full2= self.fc2(full1)
        # return full2

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, DROPOUT, PAD_IDX)

In [105]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,426,501 trainable parameters


In [106]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data.shape)

embedding.weight torch.Size([10002, 300])
convs.0.weight torch.Size([150, 300, 4])
convs.0.bias torch.Size([150])
convs.1.weight torch.Size([150, 300, 5])
convs.1.bias torch.Size([150])
conv2.weight torch.Size([100, 1, 3])
conv2.bias torch.Size([100])
fc1.weight torch.Size([100, 200])
fc1.bias torch.Size([100])
fc2.weight torch.Size([1, 100])
fc2.bias torch.Size([1])


In [107]:
print(model)
print(dir(model))

CNN1d(
  (embedding): Embedding(10002, 300, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(300, 150, kernel_size=(4,), stride=(1,))
    (1): Conv1d(300, 150, kernel_size=(5,), stride=(1,))
  )
  (conv2): Conv1d(1, 100, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=200, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)
['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_parameters', '_register_load_state_dict_pre_hook',

In [108]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.8048,  0.7626, -0.5420,  ..., -1.8786, -0.0687,  0.3894],
        [-1.5836,  0.6220, -2.2025,  ...,  1.1934,  1.0526,  0.7467],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [-0.0033, -0.2900,  0.3919,  ...,  0.2570, -0.5726, -0.2602],
        [-0.0162, -0.1061, -0.0988,  ...,  0.1458,  0.4053, -0.0294],
        [-0.4162, -0.1058, -0.3187,  ...,  0.0604, -0.1647, -0.1134]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    #Returns accuracy per batch, i.e. for 8/10 right, this returns 0.8 #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train() #to ensure the dropout is "turned on" while training.
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval() #to ensure the dropout is "turned off" while evaluating.
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [116]:
N_EPOCHS = 3
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 45s
	Train Loss: 0.559 | Train Acc: 68.79%
	 Val. Loss: 0.422 |  Val. Acc: 80.71%
Epoch: 02 | Epoch Time: 0m 45s
	Train Loss: 0.328 | Train Acc: 85.81%
	 Val. Loss: 0.398 |  Val. Acc: 83.19%
Epoch: 03 | Epoch Time: 0m 45s
	Train Loss: 0.202 | Train Acc: 92.14%
	 Val. Loss: 0.411 |  Val. Acc: 84.29%


In [69]:
model.load_state_dict(torch.load('model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.362 | Test Acc: 84.27%


In [0]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for name, layer in model.named_modules():
    layer.register_forward_hook(get_activation(name))

In [121]:
import spacy
nlp = spacy.load('en')


def extract_features(model,sentence,min_len=5):
  model.eval()
  tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
  if len(tokenized)>3000:
    tokenized=tokenized[:3000]
  if len(tokenized) < min_len:
      tokenized += ['<pad>'] * (min_len - len(tokenized))
  indexed = [TEXT.vocab.stoi[t] for t in tokenized]
  tensor = torch.LongTensor(indexed).to(device)
  tensor = tensor.unsqueeze(0)
  model(tensor)
  return activation['fc1']

  
extract_features(model,"This is undoubtedly the best best best movie ever!!!")

tensor([[-0.1962, -6.4804, -0.1389, -9.7250,  2.2781,  2.9112, -0.0829, -2.1120,
         -0.2649, -5.3577,  1.5222,  1.9277, -4.2840, -0.0882,  5.7938,  2.0542,
         -1.4710, -0.0627, -1.0962,  0.6844, -3.4132,  0.4857,  1.7672,  2.4364,
          1.3582, -1.3487,  5.5187, -5.9139, -7.0101,  3.4514,  0.9348,  9.3349,
         -6.6626,  2.5637, 10.3755, -1.8096,  0.4203,  9.5361,  1.5954,  1.8546,
         -0.8474,  0.6088, -2.1351, -0.3229,  2.0653,  2.0688, -0.7760, -1.1287,
         -0.6393, -0.4662,  2.6785,  7.3850,  4.5344, -0.8405, -1.5471, -3.2531,
          1.0198, -1.5518,  1.4121, 10.6444, -9.4700,  0.5770, -1.4240, -0.4122,
          0.8297,  0.3535, -4.6906,  8.0413, -1.3140,  8.7676,  4.3516,  2.7208,
         -1.7176,  5.3329,  0.0271, -1.5865, -4.4582,  5.3403,  0.6308, -0.1152,
          4.9711, -1.9958, -9.4701, -0.5949,  7.2818,  0.3410,  0.5714,  5.2545,
         -1.1924,  9.1059, -1.7496,  3.8955,  0.7718,  6.5278,  0.6475,  2.5976,
         -9.3027,  0.6308,  

In [0]:
def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [123]:
predict_sentiment(model, "This is undoubtedly the best best best movie ever!!!")

0.9996084570884705

In [124]:
predict_sentiment(model, "I have to give this zero stars")

0.04699946567416191

In [125]:
predict_sentiment(model, "Good at being terrible ")

0.000428651983384043